In [6]:
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split

In [19]:
class Dataset():
    DATA_DIR_NAME = 'data'
    TRAINING_IMG_FILENAME = 'train_image.pkl' 
    TRAINING_LABEL_FILENAME = 'train_label.pkl'
    TESTING_IMG_FILENAME = 'test_image.pkl'
    
    def __init__(self, validation_size=0.2):
        self.validation_size = validation_size
        self.build()
        
    def build(self):
        self.images = self.load_pickle(os.path.join(self.DATA_DIR_NAME, self.TRAINING_IMG_FILENAME))
        self.labels = self.load_pickle(os.path.join(self.DATA_DIR_NAME, self.TRAINING_LABEL_FILENAME))
        self.training_images, self.validation_images, self.training_labels, self.validation_labels = train_test_split(self.images, self.labels, test_size=self.validation_size)
        
        self.training_set = self.build_set(self.training_images, self.training_labels)
        self.validation_set = self.build_set(self.validation_images, self.validation_labels)
        self.test_set = self.load_pickle(os.path.join(self.DATA_DIR_NAME, self.TESTING_IMG_FILENAME))
        
    def build_set(self, images, labels):
        return [images, labels]
        
    def load_pickle(self, filename, mode='rb'):
        return np.array(pickle.load(open(filename, mode)))

In [21]:
d = Dataset()
print(d.training_set[0].shape, d.validation_set[0].shape)
print(d.training_set[1].shape, d.validation_set[1].shape)

(6400, 784) (1600, 784)
(6400,) (1600,)


In [20]:
class DatasetProvider():
    def __init__(self, augment=False, validation_size=0.2, batch_size=32):
        self.dataset = Dataset(validation_size)
        self.augment = augment
        self.batch_size = batch_size
    
    def training_set(self):
        for batch in self.generate_batch(self.dataset.training_set):
            yield batch
    
    def generate_batch(self, img_label_set):
        while True:
            all_idx = np.arange(len(img_label_set[0]))
            random_idx = np.random.choice(all_idx, size=self.batch_size)
            random_img = img_label_set[0][random_idx]
            random_label = img_label_set[1][random_idx]
            yield self.preprocess_batch(random_img, random_label)
            
    def preprocess_batch(self, img, label):
        

In [ ]:
class Model():
    
    def __init__(self):
        #TODO
        #initialize params and hyperparams
    
    def build(self):
        #TODO
        #define model and build model
        return model
    

In [ ]:
model = Model()
dataset_provider = DatasetProvider()

In [ ]:
model.fit_generator(dataset_provider.generate_batch())